In [1]:
import math
from collections import namedtuple
from ortools.linear_solver import pywraplp

Customer = namedtuple("Customer", ['index', 'demand', 'x', 'y'])


In [2]:
with open("./data/vrp_26_8_1", "r") as file:
    input_data = file.read()
lines = input_data.split('\n')
parts = lines[0].split()
customer_count = int(parts[0])
vehicle_count = int(parts[1])
vehicle_capacity = int(parts[2])

customers = []
for i in range(1, customer_count + 1):
    line = lines[i]
    parts = line.split()
    customers.append(Customer(i - 1, int(parts[0]), float(parts[1]), float(parts[2])))



In [3]:
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    raise Exception('Solver not created.')
x = {}
y = {}
###vehicle k travels from i to j
for i in range(customer_count):
    for j in range(customer_count):
        if i != j:
            for k in range(vehicle_count):
                x[i, j, k] = solver.BoolVar(f'x_{i}_{j}_{k}')
## i is served by k vehicle
for i in range(customer_count):
    for k in range(vehicle_count):
        y[i, k] = solver.BoolVar(f'y_{i}_{k}')
###  each customer is vsited only once 
for i in range(1, customer_count):  
    solver.Add(sum(y[i, k] for k in range(vehicle_count)) == 1)

for k in range(vehicle_count):
    for i in range(customer_count):
        solver.Add(sum(x[i, j, k] for j in range(customer_count) if i != j) == y[i, k])    ###  if vehicle k travels form i to j it should have served i
        solver.Add(sum(x[j, i, k] for j in range(customer_count) if i != j) == y[i, k])    ###  if vehicle k visits j to i it should be able to serve i

for k in range(vehicle_count):
    solver.Add(sum(customers[i].demand * y[i, k] for i in range(1, customer_count)) <= vehicle_capacity)  ### the customers demand served by veg=hicel should not exceed the capacity
for k in range(vehicle_count):
    solver.Add(sum(x[0, j, k] for j in range(1, customer_count)) == 1)  ### vehicel leaving the depot by vehicle k should be one 
for k in range(vehicle_count):
    solver.Add(sum(x[j, 0, k] for j in range(1, customer_count)) == 1)   ### same as entering the depot 

 ### position of customer i in vehicle k route 

for i in range(1, customer_count):
    for j in range(1, customer_count):
        if i != j:
            for k in range(vehicle_count):
                solver.Add(u[i, k] - u[j, k] + customer_count * x[i, j, k] <= customer_count - 1)   ### subtour constarint 

objective = solver.Sum(
    math.sqrt((customers[i].x - customers[j].x) ** 2 + (customers[i].y - customers[j].y) ** 2) * x[i, j, k]
    for i in range(customer_count) for j in range(customer_count) for k in range(vehicle_count) if i != j
)
solver.Minimize(objective)
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution found:')
    print(f'Objective value = {solver.Objective().Value()}')
    for k in range(vehicle_count):
        route = []
        for i in range(customer_count):
            for j in range(customer_count):
                if i != j and x[i, j, k].solution_value() > 0.5:
                    route.append((i, j))
        print(f'Route for vehicle {k}: {route}')
else:
    print('No optimal solution found.')
